In [1]:
import torch
from transformers import BertModel, BertTokenizer, BertConfig

tokenizer =BertTokenizer.from_pretrained('bert-base-chinese')
# config = BertConfig.from_pretrained('bert-base-chinese')
# config.update({'output_hidden_states':True})
# model = BertModel.from_pretrained('bert-base-chinese',config = config)

from transformers import AutoModel
checkpoint = 'bert-base-chinese'
model = AutoModel.from_pretrained(checkpoint)



Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [2]:
print(tokenizer.encode('自然语言处理'))
sentences = ['网络安全开发分为三个层级',
             '车辆系统层级网络安全开发',
             '车辆功能层级网络安全开发',
             '车辆零部件层级网络安全开发',
             '测试团队根据车辆网络安全目标制定测试技术要求及测试计划',
             '测试团队在网络安全团队的支持下，完成确认测试并编制测试报告',
             '在车辆确认结果的基础上，基于合理的理由，确认在设计和开发阶段识别出的所有风险均已被接受',]
test1 = tokenizer(sentences)
print(test1)
print(tokenizer('一辆汽车'))

[101, 5632, 4197, 6427, 6241, 1905, 4415, 102]
{'input_ids': [[101, 5381, 5317, 2128, 1059, 2458, 1355, 1146, 711, 676, 702, 2231, 5277, 102], [101, 6756, 6775, 5143, 5320, 2231, 5277, 5381, 5317, 2128, 1059, 2458, 1355, 102], [101, 6756, 6775, 1216, 5543, 2231, 5277, 5381, 5317, 2128, 1059, 2458, 1355, 102], [101, 6756, 6775, 7439, 6956, 816, 2231, 5277, 5381, 5317, 2128, 1059, 2458, 1355, 102], [101, 3844, 6407, 1730, 7339, 3418, 2945, 6756, 6775, 5381, 5317, 2128, 1059, 4680, 3403, 1169, 2137, 3844, 6407, 2825, 3318, 6206, 3724, 1350, 3844, 6407, 6369, 1153, 102], [101, 3844, 6407, 1730, 7339, 1762, 5381, 5317, 2128, 1059, 1730, 7339, 4638, 3118, 2898, 678, 8024, 2130, 2768, 4802, 6371, 3844, 6407, 2400, 5356, 1169, 3844, 6407, 2845, 1440, 102], [101, 1762, 6756, 6775, 4802, 6371, 5310, 3362, 4638, 1825, 4794, 677, 8024, 1825, 754, 1394, 4415, 4638, 4415, 4507, 8024, 4802, 6371, 1762, 6392, 6369, 1469, 2458, 1355, 7348, 3667, 6399, 1166, 1139, 4638, 2792, 3300, 7599, 7372, 1772, 234

In [3]:
from transformers import pipeline
unmasker = pipeline('fill-mask',model='bert-base-chinese')
unmasker('最自由民主的国家是[MASK].',top_k = 5)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[{'score': 0.28742319345474243,
  'token': 6443,
  'token_str': '谁',
  'sequence': '最 自 由 民 主 的 国 家 是 谁.'},
 {'score': 0.10349870473146439,
  'token': 8038,
  'token_str': '：',
  'sequence': '最 自 由 民 主 的 国 家 是 ：.'},
 {'score': 0.06040112301707268,
  'token': 8043,
  'token_str': '？',
  'sequence': '最 自 由 民 主 的 国 家 是 ？.'},
 {'score': 0.03125236555933952,
  'token': 2124,
  'token_str': '它',
  'sequence': '最 自 由 民 主 的 国 家 是 它.'},
 {'score': 0.03036455437541008,
  'token': 862,
  'token_str': '何',
  'sequence': '最 自 由 民 主 的 国 家 是 何.'}]

In [2]:
import torch
from torch.utils.data import DataLoader,Dataset
import os
import re
from random import sample
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from transformers import BertModel, BertTokenizer
from tqdm import tqdm

In [3]:
data_base_path = r"./imdb_test/aclImdb"
model_path = r'imdb_test/aclImdb/mode'
if not os.path.exists(model_path):
    os.mkdir(model_path)

In [9]:
class ImdbDataset(Dataset):
    def __init__(self,mode,testNumber = 10000, validNumber = 5000 ) -> None:
        super(ImdbDataset,self).__init__()

        text_path = [os.path.join(data_base_path,i) for i in ['test/neg', 'test/pos']]
        text_path.extend([os.path.join(data_base_path,i) for i in ['train/neg','train/pos']])

        if mode == 'train':
            self.total_file_path_list = []
            for i in text_path:
                self.total_file_path_list.extend([os.path.join(i,j) for j in os.listdir(i)])

        if mode=="test":
            self.total_file_path_list = []
            # 获取测试数据集，默认10000个数据
            for i in text_path:
                self.total_file_path_list.extend([os.path.join(i,j) for j in os.listdir(i)])
            self.total_file_path_list=sample(self.total_file_path_list,testNumber)
       
        if mode=="valid":
            self.total_file_path_list = []
            # 获取验证数据集，默认5000个数据集
            for i in text_path:
                self.total_file_path_list.extend([os.path.join(i,j) for j in os.listdir(i)])
            self.total_file_path_list=sample(self.total_file_path_list,validNumber)

['./imdb_test/aclImdb/test/neg',
 './imdb_test/aclImdb/test/pos',
 './imdb_test/aclImdb/train/neg',
 './imdb_test/aclImdb/train/pos']

In [ ]:
def tokenize(self,text):
    

In [10]:
class BertClassificationModel(nn.Module):
    def __init__(self,hidden_size = 768):
        super(BertClassificationModel,self).__init__()
        model_name = 'distilbert-base-uncased'
        self.tokenizer = BertTokenizer.from_pretrained(pretrained_model_name_or_path=model_name)
        self.bert = BertModel.from_pretrained(pretrained_model_name_or_path=model_name)

        for p in self.bert.parameters():
            p.required_grad = False
        self.fc = nn.Linear(hidden_size,2)

    def forward(self,batch_sentences):
        sentences_tokenizer = self.tokenizer()

AttributeError: module 'torch.nn' has no attribute 'module'

In [ ]:
def main():
    testNumber = 10000
    validNumber = 100
    batchsize = 250

    trainDatas = ImdbDataset(mode='test',testNumber=testNumber)
    validDatas = ImdbDataset(mode='valid',validNumber=validNumber)

    train_loader = torch.utils.data.Dataloader(trainDatas,batch_size = batchsize)
    val_loader = torch.utils.data.Dataloader(validDatas,batch_size = batchsize)

    epoch_num = 1

    print('training...')

    model = BertClassificationModel()